# CH-06 個股分析機器人

### 1️⃣ 安裝及匯入套件

In [ ]:
!pip install openai
!pip install yfinance==0.2.38
from openai import OpenAI, OpenAIError # 串接 OpenAI API
import yfinance as yf
import pandas as pd # 資料處理套件
import numpy as np
import datetime as dt # 時間套件
import requests
from bs4 import BeautifulSoup

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


### 2️⃣ 輸入 OpenAI API KEY

In [ ]:
import getpass # 保密輸入套件
api_key = getpass.getpass("請輸入金鑰：")
client = OpenAI(api_key = api_key) # 建立 OpenAI 物件

請輸入金鑰：··········


### 3️⃣ 取得股價資料

In [ ]:
# 從 yfinance 取得一周股價資料
def stock_price(stock_id="大盤", days = 10):
  if stock_id == "大盤":
    stock_id="^TWII"
  else:
    stock_id += ".TW"

  end = dt.date.today() # 資料結束時間
  start = end - dt.timedelta(days=days) # 資料開始時間
  # 下載資料
  df = yf.download(stock_id, start=start)

  # 更換列名
  df.columns = ['開盤價', '最高價', '最低價',
                '收盤價', '調整後收盤價', '成交量']

  data = {
    '日期': df.index.strftime('%Y-%m-%d').tolist(),
    '收盤價': df['收盤價'].tolist(),
    '每日報酬': df['收盤價'].pct_change().tolist(),
    '漲跌價差': df['調整後收盤價'].diff().tolist()
    }

  return data

print(stock_price("2330"))

[*********************100%%**********************]  1 of 1 completed
{'日期': ['2023-10-31', '2023-11-01', '2023-11-02', '2023-11-03', '2023-11-06', '2023-11-07', '2023-11-08', '2023-11-09', '2023-11-10'], '收盤價': [529.0, 528.0, 547.0, 549.0, 550.0, 555.0, 556.0, 557.0, 557.0], '每日報酬': [nan, -0.001890359168241984, 0.0359848484848484, 0.003656307129798808, 0.0018214936247722413, 0.009090909090909038, 0.0018018018018017834, 0.0017985611510791255, 0.0], '漲跌價差': [nan, -1.0, 19.0, 2.0, 1.0, 5.0, 1.0, 1.0, 0.0]}


### 4️⃣ 取得基本面資料

In [ ]:
# 基本面資料
def stock_fundamental(stock_id= "大盤"):
  if stock_id == "大盤":
      return None

  stock_id += ".TW"
  stock = yf.Ticker(stock_id)

  # 營收成長率
  quarterly_revenue_growth = np.round(stock.quarterly_financials.loc["Total Revenue"].pct_change(-1).dropna().tolist(), 2)

  # 每季EPS
  quarterly_eps = np.round(stock.quarterly_financials.loc["Basic EPS"].dropna().tolist(), 2)

  # EPS季增率
  quarterly_eps_growth = np.round(stock.quarterly_financials.loc["Basic EPS"].pct_change(-1).dropna().tolist(), 2)

  # 轉換日期
  dates = [date.strftime('%Y-%m-%d') for date in stock.quarterly_financials.columns]

  data = {
      '季日期': dates[:len(quarterly_revenue_growth)],
      '營收成長率': quarterly_revenue_growth.tolist(),
      'EPS': quarterly_eps[0:3].tolist(),
      'EPS 季增率': quarterly_eps_growth[0:3].tolist()
  }

  return data

print(stock_fundamental("2330"))

{'季日期': ['2023-09-30', '2023-06-30', '2023-03-31'], '營收成長率': [0.14, -0.05, -0.19], 'EPS': [8.14, 7.01, 7.98], 'EPS 季增率': [0.16, -0.12, -0.3]}


### 5️⃣ 取得新聞資料

In [ ]:
# 新聞資料
def stock_news(stock_name ="大盤"):
  if stock_name == "大盤":
    stock_name="台股 -盤中速報"

  data=[]
  # 取得 Json 格式資料
  json_data = requests.get(f'https://ess.api.cnyes.com/ess/api/v1/news/keyword?q={stock_name}&limit=5&page=1').json()

  # 依照格式擷取資料
  items=json_data['data']['items']
  for item in items:
      # 網址、標題和日期
      news_id = item["newsId"]
      title = item["title"]
      publish_at = item["publishAt"]
      # 使用 UTC 時間格式
      utc_time = dt.datetime.utcfromtimestamp(publish_at)
      formatted_date = utc_time.strftime('%Y-%m-%d')
      # 前往網址擷取內容
      url = requests.get(f'https://news.cnyes.com/'
                        f'news/id/{news_id}').content
      soup = BeautifulSoup(url, 'html.parser')
      p_elements=soup .find_all('p')
      # 提取段落内容
      p=''
      for paragraph in p_elements[4:]:
          p+=paragraph.get_text()
      data.append([stock_name, formatted_date ,title,p])
  return data

print(stock_news("台積電"))

[['台積電', '2023-11-10', '<mark>台積電</mark>:台積公司2023年10月營收報告', '第51款1.事實發生日:112/11/102.公司名稱:台灣積體電路製造股份有限公司3.與公司關係(請輸入本公司或子公司):本公司4.相互持股比例:不適用5.發生緣由:不適用6.因應措施:不適用7.其他應敘明事項(若事件發生或決議之主體係屬公開發行以上公司，本則重大訊息同時  符合證券交易法施行細則第7條第9款所定對股東權益或證券價格有重大影響之事項):台積公司今（10）日公佈2023年10月營收報告。2023年10月合併營收約為新台幣2,432億300萬元，較上月增加了34.8%，較去年同期增加了15.7%。累計2023年1至10月營收約為新台幣1兆7,794億1,000萬元，較去年同期減少了3.7%。'], ['台積電', '2023-11-10', '〈台股盤後〉跌62點收16682點 失守五日線周線連二紅', '受到美國聯準會 (Fed) 主席鮑爾態度偏鷹、美債殖利率走高影響，美股收黑，亞股全盤皆墨，台股今 (10) 日同步開低震盪，加上電子三雄疲軟，金融、傳產撐盤無力，指數失守 16700 點關卡、5 日線，終場收在 16682.67 點，下跌 62.98 點或 0.38%，成交量 2302.38 億元，日 K 線終止連七紅、周線連二紅。電子權值股普遍疲軟，台積電 (2330-TW) 終場以平盤作收；聯發科 (2454-TW) 震盪走低，收在今日最低點 884 元，下跌逾 2%；鴻海也收黑，下跌約 0.7%；廣達、聯電、聯詠、研華等都表現疲軟。半導體群族仍有零星點火，金麗科強漲 7%，安國、沛亨也有逾 3% 漲幅；穩懋、點序、全宇昕、中美晶、旺矽、普誠、欣銓、合晶等逆勢收盤上。另外，精元、迎廣、勤誠、倫飛、圓剛、神達等表現也不俗。生技股上沖下洗，合一 (4743-TW) 開發糖尿病族足部傷口潰瘍新藥獲得中國認證，股價衝上漲停後賣壓出籠，終場翻黑下跌逾 4%，同集團的中天跌幅收斂，小漲作收。飯店股漲勢整齊，寒舍亮燈漲停，富野拉出逾根停板、力麗店、洛碁、亞都都有 2-3% 漲幅；老爺知、遠雄來、第一店、雲品同步收紅；另外，觀光族群的五福飆漲逾 9%，燦星旅、富野、易飛網、山富、豆府表現也不俗。傳產方面，中鋼 10 月自結稅前盈餘 8.23 

### 6️⃣ 爬取股號、股名對照表

In [ ]:
# 取得全部股票的股號、股名
def stock_name():
  print("線上讀取股號、股名、及產業別")

  response = requests.get('https://isin.twse.com.tw/isin/C_public.jsp?strMode=2')
  url_data = BeautifulSoup(response.text, 'html.parser')
  stock_company = url_data.find_all('tr')

  # 資料處理
  data = [
      (row.find_all('td')[0].text.split('\u3000')[0].strip(),
        row.find_all('td')[0].text.split('\u3000')[1],
        row.find_all('td')[4].text.strip())
      for row in stock_company[2:] if len(row.find_all('td')[0].text.split('\u3000')[0].strip()) == 4
  ]

  df = pd.DataFrame(data, columns=['股號', '股名', '產業別'])

  return df

name_df = stock_name()

線上讀取股號、股名、及產業別


### 7️⃣ 取得股票名稱

In [ ]:
# 取得股票名稱
def get_stock_name(stock_id, name_df):
    return name_df.set_index('股號').loc[stock_id, '股名']

print(name_df.head())
print("--------------------------")
print(get_stock_name("1417",name_df))

     股號  股名   產業別
0  1101  台泥  水泥工業
1  1102  亞泥  水泥工業
2  1103  嘉泥  水泥工業
3  1104  環泥  水泥工業
4  1108  幸福  水泥工業
--------------------------
嘉裕


### 8️⃣ 建構 GPT 3.5 模型

In [ ]:
# 建立 GPT 3.5-16k 模型
def get_reply(messages):
    try:
        response = client.chat.completions.create(
            model = "gpt-3.5-turbo",
            messages = messages
        )
        reply = response.choices[0].message.content
    except OpenAIError as err:
        reply = f"發生 {err.type} 錯誤\n{err.message}"
    return reply

# 建立訊息指令(Prompt)
def generate_content_msg(stock_id, name_df):

    stock_name = get_stock_name(
        stock_id, name_df) if stock_id != "大盤" else stock_id

    price_data = stock_price(stock_id)
    news_data = stock_news(stock_name)

    content_msg = f'請依據以下資料來進行分析並給出一份完整的分析報告:\n'

    content_msg += f'近期價格資訊:\n {price_data}\n'

    if stock_id != "大盤":
        stock_value_data = stock_fundamental(stock_id)
        content_msg += f'每季營收資訊：\n {stock_value_data}\n'

    content_msg += f'近期新聞資訊: \n {news_data}\n'
    content_msg += f'請給我{stock_name}近期的趨勢報告,請以詳細、\
      嚴謹及專業的角度撰寫此報告,並提及重要的數字, reply in 繁體中文'

    return content_msg

# StockGPT
def stock_gpt(stock_id, name_df=name_df):
    content_msg = generate_content_msg(stock_id, name_df)

    msg = [{
        "role": "system",
        "content": f"你現在是一位專業的證券分析師, 你會統整近期的股價\
      、基本面、新聞資訊等方面並進行分析, 然後生成一份專業的趨勢分析報告"
    }, {
        "role": "user",
        "content": content_msg
    }]

    reply_data = get_reply(msg)

    return reply_data


### 9️⃣ 大盤趨勢報告

In [ ]:
reply = stock_gpt(stock_id="大盤")
print(reply)

[*********************100%%**********************]  1 of 1 completed
根據提供的股價資訊和新聞資訊，以下是對大盤近期趨勢的分析報告：

一、股價走勢：
自2023年10月31日至11月10日，台股加權指數呈現整體上漲的走勢。10月31日，台股收盤價為16001.27點，隨後於11月2日達到最高峰，收盤價達到16396.95點。然而，在11月10日，股價出現小幅調整，收盤價為16682.67點，較前一交易日下跌0.38%。從每日報酬率的變化來看，股價整體上呈現正向波動，但在11月10日出現了負的日報酬率，顯示市場出現了一定程度的不確定性。

二、新聞資訊影響：
根據近期新聞報導，美國聯準會主席鮑爾的談話以及美國債券殖利率的走高對台股產生了一定影響。個股方面，電子權值股整體疲軟，影響了台股指數的表現。此外，影響力較大的概念股包括AI和生技股。其中，AI概念股中的PCB和生技股合一在新產品認證以及公告中表現強勁。此外，觀光股和部分傳產股也呈現一定的強勢表現。這些消息對股價產生了一定程度的波動。

三、技術面分析：
從技術面來看，股價在5日線附近震蕩，並未能持續突破，此外，當日成交量也較前幾個交易日出現了下降。這可能意味著投資者對於市場的信心有所動搖，有一定的觀望情緒存在。

總結來看，從股價趨勢和新聞資訊來看，台股在短期內呈現整體上升的走勢，同時也存在一定的波動性。技術面表現較為脆弱，投資者在操作中需要謹慎


### 🔟 個股分析報告

In [ ]:
reply = stock_gpt(stock_id="2330")
print(reply)

[*********************100%%**********************]  1 of 1 completed
【台積電近期趨勢分析報告】

一、價格資訊分析:
   近期台積電股價呈現整體上漲趨勢，收盤價從529元一路攀升至557元。每日報酬率亦顯示出股價整體上揚的趨勢，從-0.19%上升至0%。
   
二、基本面分析:
   1. 季度營收成長率顯示，台積電在2023年9月份取得了驚人的1136.03%成長率，呈現出強勁的業績表現。
   2. 另外，每股盈餘(EPS)從2022年12月的11.41元持續下降，呈現逐步減少的趨勢。然而，2023年9月的EPS為8.14元，還是呈現相對良好的表現。然而，季度增長率顯示EPS正逐漸下降。

三、新聞資訊分析:
   1. 台積電在新聞中公佈2023年10月營收約為2,432億300萬元，較上月增加了34.8%，較去年同期增加了15.7%。累計2023年1至10月營收約為1兆7,794億1,000萬元，較去年同期減少了3.7%。此營收數據顯示，台積電的營收整體呈現出正向成長的趨勢。
   2. 受到美國聯準會 (Fed) 主席鮑爾態度偏鷹、美債殖利率走高影響，全球股市普遍承壓。然而，台積電在此情況下仍取得驚人的10月營收數據，顯示其良好的市場表現。
   3. 11月除息ETF中，有16檔台股ETF進行除息，其中對於台積電的未來投資策略具有一定的參考價值。
   
四、趨勢分析總結:
   從股價、基本面及新聞資訊的綜合分析來看，台積電整體呈現出穩健的營收增長及穩定的股價上升趨勢。然而，EPS的逐步下降值得投資者關注。受到宏觀經濟因素的影響，投資者需密切留意全球股市動態，並謹慎評估其風險。

以上報告僅供參考，投資者在做出交易決策前，還應該考慮個人投資風險偏好及其他市場因素，以及尋求相關專業建議。


### 1️⃣1️⃣ 雞蛋水餃股也能做分析

In [ ]:
reply = stock_gpt(stock_id="4414")
print(reply)

[*********************100%%**********************]  1 of 1 completed
## 如興公司近期趨勢報告

### 股價走勢分析
- 根據2023年11月10日的收盤價（3.63元），相較於10月31日的收盤價（3.46元），股價出現上漲趨勢。具體數據顯示11月10日的收盤價較10月31日上漲0.17元，漲幅為4.91%。
- 在過去10個交易日中，如興股價的每天報酬率波動明顯。例如，11月9日與11月7日的報酬率分別為-1.66%和2.28%。這顯示了股價的波動性較大。

### 基本面分析
- 依據最近的營收情況來看，公司在2023年第三季度實現了正成長，營收增長率為11%，顯示公司營運有穩定向好的發展。
- 如興公司EPS（每股收益）也出現了逐季改善的趨勢，由去年底-1.36元，逐步改善至最新的-0.31元，顯示公司在盈利能力上有所提升。同時，觀察到EPS季增率出現正數，這意味著公司的盈利能力有所改善。

### 新聞事件分析
- 10月26日，如興公告宣布進行私募認股，募集資金來源明細顯示為貸款63,755,000元，這將對公司未來的財務結構及股東權益產生一定影響。
- 公司最近一次公告中提到興牛一投資股份有限公司持股總額占公司已發行股份總額百分比為10.73%，這顯示了少數股東的機構投資者對公司的未來發展抱有信心。

### 總結
綜合股價走勢、基本面和最近新聞事件的分析，我們可以看到如興公司股價整體呈現波動明顯的特點，且公司營收及盈利狀況有穩步改善的趨勢。在未來投資中，投資者需要密切關注公司的財務狀況及市場動態，以做出明智的投資決策。

以上報告僅為個人觀點，請自行謹慎評估風險並考量專業投資意見。
